In [5]:
### Imports
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import datetime
import matplotlib.pyplot as plt 
import pydicom 
import pydicom.data 
from pathlib import Path
import gc
import warnings
warnings.filterwarnings("ignore")
import sys
import time
####
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
####
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim
import torchvision.transforms as transforms
import torchvision
##
from fastprogress import master_bar, progress_bar
from PIL import Image
from tqdm import tqdm
gc.collect()

30

In [6]:
full_train_df = pd.read_csv(Path('image_labels_train.csv'))
full_train_df.head()

,image_id,rad_ID,No finding,Bronchitis,Brocho-pneumonia,Other disease,Bronchiolitis,Situs inversus,Pneumonia,Pleuro-pneumonia,Diagphramatic hernia,Tuberculosis,Congenital emphysema,CPAM,Hyaline membrane disease,Mediastinal tumor,Lung tumor
0,6cb53aff85c71b98ad13d67a131708c6,R3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40414c05687cdb156823c156967b13f0,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0e4a464dfbf8abc6333c82f1b77b6455,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,f4d3fab0b71381e6b237dc36301e85a0,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,b58c9b1c89978a0b1f8533b7a2ca1088,R3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
num_classes = 15

# Load the CSV file containing image labels
full_train_df = pd.read_csv('image_labels_train.csv')

# Create an empty list for image data and labels
x_train = []
y_train = []

# Specify the folder directory containing the image files
folder_dir = 'C:/Users/HP/Desktop/Untitled Folder/train/'

# Define a transform to preprocess the images (adjust this as needed)
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to 32x32 pixels
    transforms.ToTensor()  # Convert to a torch tensor
])

# Get all image paths and image labels from the dataframe
for index, row in tqdm(full_train_df.iterrows(), total=len(full_train_df)):
    image_path = folder_dir + str(row.image_id) + '.dicom'
    dataset = pydicom.dcmread(image_path, force=True)
    pil_image_data = Image.fromarray(dataset.pixel_array, mode="L")
    
    pil_image_data = pil_image_data.convert('RGB')
    
    # Apply the defined transform to the image
    image_data = transform(pil_image_data)
    x_train.append(image_data)
    
    # Create one-hot encoded labels for each class
    label = 2
    for col in row[2:]:
        if col == 1:
            label = col
            break
    y_train.append(label)

# Convert the lists to numpy arrays
x_train = np.stack(x_train, axis=0)
y_train = np.array(y_train, dtype=int)

# Reshape the labels to match Keras format
y_train = y_train.reshape(y_train.shape[0], 1)
x_train = x_train.reshape(x_train.shape[0],32,32,3)

# Print the shapes of the loaded data
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")

100%|██████████| 7728/7728 [11:01<00:00, 11.68it/s]

x_train shape: (7728, 32, 32, 3) - y_train shape: (7728, 1)


In [36]:
num_classes = 15

# Load the CSV file containing image labels
full_test_df = pd.read_csv('image_labels_test.csv')

# Create an empty list for image data and labels
x_test = []
y_test = []

# Specify the folder directory containing the image files
folder_dir = 'C:/Users/HP/Desktop/Untitled Folder/test/'

# Define a transform to preprocess the images (adjust this as needed)
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to 32x32 pixels
    transforms.ToTensor()  # Convert to a torch tensor
])

# Get all image paths and image labels from the dataframe
for index, row in tqdm(full_test_df.iterrows(), total=len(full_test_df)):
    image_path = folder_dir + str(row.image_id) + '.dicom'
    dataset = pydicom.dcmread(image_path, force=True)
    pil_image_data = Image.fromarray(dataset.pixel_array, mode="L")
    
    pil_image_data = pil_image_data.convert('RGB')
    
    # Apply the defined transform to the image
    image_data = transform(pil_image_data)
    x_test.append(image_data)
    
    # Create one-hot encoded labels for each class
    label = 2
    for col in row[2:]:
        if col == 1:
            label = col
            break
    y_test.append(label)

# Convert the lists to numpy arrays
x_test = np.stack(x_test, axis=0)
y_test = np.array(y_test, dtype=int)

# Reshape the labels to match Keras format
y_test = y_test.reshape(y_test.shape[0], 1)
x_test = x_test.reshape(x_test.shape[0],32,32,3)

# Print the shapes of the loaded data
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

100%|██████████| 1397/1397 [02:06<00:00, 11.00it/s]

x_test shape: (1397, 32, 32, 3) - y_test shape: (1397, 1)


In [24]:
pip install tensorflow-addons

Note: you may need to restart the kernel to use updated packages.


In [30]:
num_classes = 15
input_shape = (32,32,3)

In [31]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [32]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

In [42]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="dl-encoder")
    return model


encoder = create_encoder()
encoder.summary()

learning_rate = 0.001
batch_size = 100
hidden_units = 512
projection_units = 128
num_epochs = 10
dropout_rate = 0.5
temperature = 0.05

Model: "dl-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 sequential_1 (Sequential)   (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23564807 (89.89 MB)
Trainable params: 23519360 (89.72 MB)
Non-trainable params: 45447 (177.53 KB)
_________________________________________________________________


In [43]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="dl-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [44]:
encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)


Model: "dl-classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 dl-encoder (Functional)     (None, 2048)              23564807  
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_5 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 15)                7695      
                                                                 
Total params: 24621590 (93.92 MB)
Trainable params: 2

In [45]:
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

44/44 [==============================] - 4s 74ms/step - loss: 0.0500 - sparse_categorical_accuracy: 0.9928
Test accuracy: 99.28%


In [46]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="dl-encoder_with_projection-head"
    )
    return model

In [47]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
)

Model: "dl-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 dl-encoder (Functional)     (None, 2048)              23564807  
                                                                 
 dense_7 (Dense)             (None, 128)               262272    
                                                                 
Total params: 23827079 (90.89 MB)
Trainable params: 23781632 (90.72 MB)
Non-trainable params: 45447 (177.53 KB)
_________________________________________________________________
Epoch 1/10
78/78 [==============================] - 174s 2s/step - loss: 4.6817
Epoch 2/10
78/78 [==============================] - 165s 2s/step - loss: 4.6003
Epoch 3/10
78/78 [==============================] - 163s 2s/step - loss: 4.6004
Epoch 4/10
78

In [48]:
classifier = create_classifier(encoder, trainable=False)
history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

Epoch 1/10
78/78 [==============================] - 15s 164ms/step - loss: 0.2609 - sparse_categorical_accuracy: 0.9801
Epoch 2/10
78/78 [==============================] - 13s 164ms/step - loss: 0.1643 - sparse_categorical_accuracy: 0.9930
Epoch 3/10
78/78 [==============================] - 13s 164ms/step - loss: 0.1302 - sparse_categorical_accuracy: 0.9929
Epoch 4/10
78/78 [==============================] - 13s 166ms/step - loss: 0.2459 - sparse_categorical_accuracy: 0.9928
Epoch 5/10
78/78 [==============================] - 13s 163ms/step - loss: 0.2024 - sparse_categorical_accuracy: 0.9930
Epoch 6/10
78/78 [==============================] - 13s 165ms/step - loss: 0.1439 - sparse_categorical_accuracy: 0.9926
Epoch 7/10
78/78 [==============================] - 13s 165ms/step - loss: 0.1003 - sparse_categorical_accuracy: 0.9926
Epoch 8/10
78/78 [==============================] - 13s 164ms/step - loss: 0.1525 - sparse_categorical_accuracy: 0.9929
Epoch 9/10
78/78 [======================

In [49]:
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

44/44 [==============================] - 4s 74ms/step - loss: 1.6782 - sparse_categorical_accuracy: 0.9928
Test accuracy: 99.28%
